# Creating a netCDF file with eNd for atmospheric flux- distinct eNd boxes
netCDF file on a FAMOUS grid, containing eNd values across the ocean for the atmospheric flux
File produced contains 5 variables with each distinct atmospheric Nd box 

setting whole ocean to a background eNd value of -8.0 

Following Rempfer et al. 2011 dust boxes

North Atlantic >50 °N eNd = -15
Atlantic < or equal to 50 °N eNd = -12
North Pacific >44 °N eNd = -5
Indopacific < or equal to 44 °N eNd = -7
Remainder eNd= -8

creating a .nc file which i will translate into an ancil file 
from which I will create a mod to convert this into a dust flux of 144Nd and 143Nd

based on examples from:
https://iescoders.com/2017/10/03/writing-netcdf4-data-in-python/
http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/11_create_netcdf_python.pdf

In [1]:
# load packages

%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend
import matplotlib.gridspec as gridspec # subplots
from matplotlib import colors as mcolors

import iris
import iris.plot as iplt
import iris.quickplot as qplt
#plt.interactive(True)

import numpy as np
import pandas as pd

import cf_units as cf


## Importing a salinity netCDF and writing to a new netCDF with dust flux boxes

In [2]:
from netCDF4 import Dataset

#this is an nc file with salinity from which to base my template on
f=Dataset('/nfs/see-fs-01_users/ee14s2r/scripts/python/surf_flux/qrparm.waterfix.famous.nc','r')

#latitude in and longitude in
latin = f.variables['latitude'][:]
latsize=len(latin)
lonin = f.variables['longitude'][:]
lonsize=len(lonin)
#time=f.variables['t'][:]

#if xconv qrparm.waterfix.famous.nc then it will have the field number
waterflux=f.variables['field672'][:]


#Setting the background to nan
waterflux2=np.where(waterflux>1.0E20,np.nan, -8.0)



#open a new netCDF file to create my file for inputing 144Nd surface flux
dataset= Dataset('/nfs/see-fs-01_users/ee14s2r/working/surf_files/eNd_dust_mask/dust_boxes_eNd.nc', 'w',
                 format='NETCDF4_CLASSIC')

print(dataset.file_format)
print(waterflux2)
print(np.shape(waterflux2))

NETCDF4_CLASSIC
[[[[ nan  nan  nan ...,  nan  nan  nan]
   [ nan  nan  nan ...,  nan  nan  nan]
   [ nan  nan  nan ...,  nan  nan  nan]
   ..., 
   [ -8.  -8.  -8. ...,  -8.  -8.  -8.]
   [ -8.  -8.  -8. ...,  -8.  -8.  -8.]
   [ nan  nan  nan ...,  nan  nan  nan]]]]
(1, 1, 73, 98)


In [3]:
print(lonin)

[   0.      3.75    7.5    11.25   15.     18.75   22.5    26.25   30.
   33.75   37.5    41.25   45.     48.75   52.5    56.25   60.     63.75
   67.5    71.25   75.     78.75   82.5    86.25   90.     93.75   97.5
  101.25  105.    108.75  112.5   116.25  120.    123.75  127.5   131.25
  135.    138.75  142.5   146.25  150.    153.75  157.5   161.25  165.
  168.75  172.5   176.25  180.    183.75  187.5   191.25  195.    198.75
  202.5   206.25  210.    213.75  217.5   221.25  225.    228.75  232.5
  236.25  240.    243.75  247.5   251.25  255.    258.75  262.5   266.25
  270.    273.75  277.5   281.25  285.    288.75  292.5   296.25  300.
  303.75  307.5   311.25  315.    318.75  322.5   326.25  330.    333.75
  337.5   341.25  345.    348.75  352.5   356.25  360.    363.75]


In [4]:
print(latin)

[-90.  -87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5
 -60.  -57.5 -55.  -52.5 -50.  -47.5 -45.  -42.5 -40.  -37.5 -35.  -32.5
 -30.  -27.5 -25.  -22.5 -20.  -17.5 -15.  -12.5 -10.   -7.5  -5.   -2.5
   0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5
  90. ]


In [5]:
#box=(latin>0.0)&(latin<40.0)&(lonin>145.5)&(lonin<215.25)


In [6]:
#Define dimensions used for my variables
depth = dataset.createDimension("depth", 1)
lat = dataset.createDimension("lat", latsize)
lon = dataset.createDimension("lon", lonsize)
time = dataset.createDimension("time", 1)

In [7]:
#Checking dimensions
print (len(lon))

98


## Creating variables with two dimensions (lon and lat) 

In [8]:
#Creating a variable
depth = dataset.createVariable("depth", np.int32, ("depth",))
depth.units = 'm'
depth.positive= 'down'
lat = dataset.createVariable("lat", np.float32, ("lat",))
lat.units = 'degrees_north'
lat.point_spacing = 'even'
lat.long_name = 'latitude'
lon = dataset.createVariable("lon", np.float32, ("lon",))
lon.units = 'degrees_east'
lon.point_spacing = 'even'
lon.long_name = 'longitude'
time = dataset.createVariable("time", np.float64, ("time",))
time.units = ('days since 0000-01-01 00:00.00') 

In [9]:
#writing data and slicing it to lon and lat
lat[:]= latin
lon[:]= lonin

print(lat[1])

-87.5


## Creating variable for Background eNd (-8) and setting nan for the land mask 

In [10]:
#This np.where is changing values of salnity into background eNd values and then changing the land mask to 0
waterflux3=np.where(waterflux>1.0E20,np.nan,-8.0)

#Create the 2d-variable for backgrond and land mask
background = dataset.createVariable("background", np.float32, ("time","depth","lat","lon"))
#background.units = "ratio"
background.long_name = "background_Epsilon_Nd"

#filling the background eNd values
background[:,:,:,:]=waterflux3

In [11]:
print(background[0,0,:,:])

[[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ -8.  -8.  -8. ...,  -8.  -8.  -8.]
 [ -8.  -8.  -8. ...,  -8.  -8.  -8.]
 [ nan  nan  nan ...,  nan  nan  nan]]


## Creating variable for North Atlantic Box

In [12]:
#Create the 2d-variable for N.Atlantix box 
N_Atlantic = dataset.createVariable("N_Atlantic", np.float32, ("time","depth","lat","lon"))
#N_Atlantic.units = "epsilon"
N_Atlantic.long_name = "N_Atlantic_Epsilon_Nd"
print(N_Atlantic)

#filling the N.Atlantic with nan
N_Atlantic[:,:,:,:]=waterflux2



<type 'netCDF4._netCDF4.Variable'>
float32 N_Atlantic(time, depth, lat, lon)
    long_name: N_Atlantic_Epsilon_Nd
unlimited dimensions: 
current shape = (1, 1, 73, 98)
filling on, default _FillValue of 9.96920996839e+36 used



In [13]:

#setting the boundaries for North Atlantic box and setting the eNd value

#Assuming the top of greenland is 83 degrees North, then above that is the Arctic Ocean  
NA_lat_bounds1 = np.logical_and(latin  >= 50.0, latin <= 80.0) 
NA_lon_bounds1 = np.logical_and(lonin  >= 270.0, lonin < 356.25) 


#Across England and Scandanavia
NA_lat_bounds2 = np.logical_and(latin  >= 50.0, latin <= 80.0) 
NA_lon_bounds2 = np.logical_and(lonin  >= 356.25, lonin <= 363.75)

#Across right of Scandanavia
NA_lat_bounds3 = np.logical_and(latin  >= 50.0, latin <= 80.0) 
NA_lon_bounds3 = np.logical_and(lonin  >= 0.0, lonin <= 18.75)

#Small box in tge top left inAlaska 
NA_lat_bounds4 = np.logical_and(latin  >= 50.0, latin <= 70.0) 
NA_lon_bounds4 = np.logical_and(lonin  >= 266.25, lonin <= 270.0)

#Setting to -15.0
N_Atlantic[:,:,NA_lat_bounds1,NA_lon_bounds1]=-15.0
N_Atlantic[:,:,NA_lat_bounds2,NA_lon_bounds2]=-15.0
N_Atlantic[:,:,NA_lat_bounds3,NA_lon_bounds3]=-15.0
N_Atlantic[:,:,NA_lat_bounds4,NA_lon_bounds4]=-15.0

In [14]:
#Checking that it is not including 50.0 degrees latitude
#print different values to see wheneach dimension makes the lat or lon i want to look at
# ie (latin[56]= 50 degrees north)
print(latin[56])
print(lonin[80])

print(N_Atlantic[:,:,57,80])

50.0
300.0
[[-15.]]


## Creating variable for Atlantic Box 

In [15]:
#Create the 2d-variable for N.Atlantix box 
Atlantic = dataset.createVariable("Atlantic", np.float32, ("time","depth","lat","lon"))
#Atlantic.units = "epsilon"
Atlantic.long_name = "Atlantic_Epsilon_Nd"
print(Atlantic)

#filling the Atlantic with nan
Atlantic[:,:,:,:]=waterflux2

<type 'netCDF4._netCDF4.Variable'>
float32 Atlantic(time, depth, lat, lon)
    long_name: Atlantic_Epsilon_Nd
unlimited dimensions: 
current shape = (1, 1, 73, 98)
filling on, default _FillValue of 9.96920996839e+36 used



In [16]:
#setting the boundaries for Atlantic box and setting the eNd value

#Bottom of England to the top of Africa
A_lat_bounds1 = np.logical_and(latin  >= 25.0, latin <= 50.0) 
A_lon_bounds1 = np.logical_and(lonin  > 270.0, lonin < 360.0) 

#Top of Africa, down to middle of South America
A_lat_bounds2 = np.logical_and(latin  >= -35.0, latin < 25.0) 
A_lon_bounds2 = np.logical_and(lonin  >= 292.5, lonin <= 363.75) 

#Reaching to the bottom tip of Africa
A_lat_bounds3 = np.logical_and(latin  >= -35.0, latin <= 15.0) 
A_lon_bounds3 = np.logical_and(lonin  >= 0.0, lonin <= 18.75) 

#Central America
#The lower bound for latin -> check when on FAMOUS grid as it is very close!!!
A_lat_bounds4 = np.logical_and(latin  >=10.0, latin <= 22.5) 
A_lon_bounds4 = np.logical_and(lonin  > 270.0, lonin <= 288.75)

#North America - tiny box
A_lat_bounds5 = np.logical_and(latin  >=17.5, latin <= 30.0) 
A_lon_bounds5 = np.logical_and(lonin  >= 258.75, lonin <= 270.0)

Atlantic[:,:,A_lat_bounds1,A_lon_bounds1]=-12.0
Atlantic[:,:,A_lat_bounds2,A_lon_bounds2]=-12.0
Atlantic[:,:,A_lat_bounds3,A_lon_bounds3]=-12.0
Atlantic[:,:,A_lat_bounds4,A_lon_bounds4]=-12.0
Atlantic[:,:,A_lat_bounds5,A_lon_bounds5]=-12.0
#print(Atlantic[:,:,56,80])

## Creating variable for North Pacific box 

In [17]:
#Create the 2d-variable for N.Pacific box 
N_Pacific = dataset.createVariable("N_Pacific", np.float32, ("time","depth","lat","lon"))
#N_Pacific.units = "epsilon"
N_Pacific.long_name = "N_Pacific_Epsilon_Nd"
print(N_Pacific)

#filling the pacific with nan
N_Pacific[:,:,:,:]=waterflux2

#print(np.shape(N.Pacific))
#print(np.shape(waterflux2))
#print(N.Pacific)
#print(N.Pacific[:,:,:,:])

<type 'netCDF4._netCDF4.Variable'>
float32 N_Pacific(time, depth, lat, lon)
    long_name: N_Pacific_Epsilon_Nd
unlimited dimensions: 
current shape = (1, 1, 73, 98)
filling on, default _FillValue of 9.96920996839e+36 used



In [18]:
#setting the boundaries for North Pacific box and setting the eNd value

#Assuming the Artic ocean starts at 65.25 degees North
NP_lat_bounds1 = np.logical_and(latin  >= 45.0, latin <= 67.5) 
NP_lon_bounds1 = np.logical_and(lonin  >= 120.0, lonin <= 262.5) 

N_Pacific[:,:,NP_lat_bounds1,NP_lon_bounds1]=-5.0


In [19]:
print(N_Pacific[:,:,:,:])

[[[[ nan  nan  nan ...,  nan  nan  nan]
   [ nan  nan  nan ...,  nan  nan  nan]
   [ nan  nan  nan ...,  nan  nan  nan]
   ..., 
   [ -8.  -8.  -8. ...,  -8.  -8.  -8.]
   [ -8.  -8.  -8. ...,  -8.  -8.  -8.]
   [ nan  nan  nan ...,  nan  nan  nan]]]]


## Creating a variable for the IndoPacific 

In [20]:
#Create the 2d-variable for IndoPacific box 
I_Pacific = dataset.createVariable("I_Pacific", np.float32, ("time","depth","lat","lon"))
#I_Pacific.units = "epsilon"
I_Pacific.long_name = "Indo-Pacific_Epsilon_Nd"
print(I_Pacific)

#filling the Indo-Pacific with nan
I_Pacific[:,:,:,:]=waterflux2

<type 'netCDF4._netCDF4.Variable'>
float32 I_Pacific(time, depth, lat, lon)
    long_name: Indo-Pacific_Epsilon_Nd
unlimited dimensions: 
current shape = (1, 1, 73, 98)
filling on, default _FillValue of 9.96920996839e+36 used



In [21]:
#setting the boundaries for Indo-Pacific box and setting the eNd value

#Remfper did to 44 lat but in FAMOUS the grid boxes go from 42.5 N to 45N
IP_lat_bounds1 = np.logical_and(latin  >= -35.0, latin <= 45.0) 
IP_lon_bounds1 = np.logical_and(lonin  >= 120.0, lonin < 262.5) 

IP_lat_bounds2 = np.logical_and(latin  >= -35.0, latin <= 30.0) 
IP_lon_bounds2 = np.logical_and(lonin  >= 30.0, lonin <= 116.25) 

IP_lat_bounds3 = np.logical_and(latin  >= -35.0, latin <= 5.0) 
IP_lon_bounds3 = np.logical_and(lonin  >= 262.5, lonin <= 292.5) 

IP_lat_bounds4 = np.logical_and(latin  >= 7.5, latin <= 15.0) 
IP_lon_bounds4 = np.logical_and(lonin  >= 262.5, lonin <= 273.75) 

IP_lat_bounds5 = np.logical_and(latin  >= 7.5, latin < 10.0) 
IP_lon_bounds5 = np.logical_and(lonin  >= 277.5, lonin <= 281.25) 

I_Pacific[:,:,IP_lat_bounds1,IP_lon_bounds1]=-7.0
I_Pacific[:,:,IP_lat_bounds2,IP_lon_bounds2]=-7.0
I_Pacific[:,:,IP_lat_bounds3,IP_lon_bounds3]=-7.0
I_Pacific[:,:,IP_lat_bounds4,IP_lon_bounds4]=-7.0
I_Pacific[:,:,IP_lat_bounds5,IP_lon_bounds5]=-7.0

In [22]:
#Accessing and checking all the variables
for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape #, var.units

depth int32 (u'depth',) (1,)
lat float32 (u'lat',) (73,)
lon float32 (u'lon',) (98,)
time float64 (u'time',) (1,)
background float32 (u'time', u'depth', u'lat', u'lon') (1, 1, 73, 98)
N_Atlantic float32 (u'time', u'depth', u'lat', u'lon') (1, 1, 73, 98)
Atlantic float32 (u'time', u'depth', u'lat', u'lon') (1, 1, 73, 98)
N_Pacific float32 (u'time', u'depth', u'lat', u'lon') (1, 1, 73, 98)
I_Pacific float32 (u'time', u'depth', u'lat', u'lon') (1, 1, 73, 98)


In [23]:
dataset.close()